### Transfer learning / Keras

In [11]:
from keras.optimizers import RMSprop

from keras.applications import inception_v3, resnet50, VGG16, MobileNet
from keras.models import Model
from keras.layers import (
    Input, Dense, 
    Dropout, BatchNormalization,
    Flatten)

from albumentations import (
    Compose, ShiftScaleRotate
)

from h5imagegenerator import HDF5ImageGenerator

In [12]:
base = resnet50.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [13]:
base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [14]:
class Head:
    @staticmethod
    def build(base, num_classes):
        head = base.output
        
        head = Flatten()(head)
        head = Dense(256, activation='relu')(head)
        head = BatchNormalization()(head)
        head = Dropout(0.5)(head)
        
        head = Dense(num_classes, activation='softmax')(head)
        
        return head

In [15]:
head = Head.build(base, num_classes=175)

In [16]:
model = Model(inputs=base.input, outputs=head)

In [17]:
for layer in base.layers:
    layer.trainable = False

In [18]:
num_epochs = 25
batch_size = 32

In [19]:
aug = Compose([
    ShiftScaleRotate(
        shift_limit=0.025,
        scale_limit=0.06,
        rotate_limit=20,
    )
])

train_gen = HDF5ImageGenerator(
    src= 'datasets/birds/train.h5',
    scaler=True,
    num_classes=175,
    batch_size=batch_size,
    augmenter=aug,
)

val_gen = HDF5ImageGenerator(
    src= 'datasets/birds/test.h5',
    scaler=True,
    num_classes=175,
    batch_size=batch_size,
)

In [20]:
rms = RMSprop(lr=0.001, rho=0.9)

callbacks = []

model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer=rms
)

h = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    workers=10,
    use_multiprocessing=True,
    verbose=1,
    epochs=num_epochs
)

Epoch 1/25
636/636 [==============================] - 102s 161ms/step - loss: 2.1645 - accuracy: 0.5756 - val_loss: 5.2997 - val_accuracy: 0.0043
Epoch 2/25
636/636 [==============================] - 89s 140ms/step - loss: 0.6815 - accuracy: 0.8779 - val_loss: 5.6019 - val_accuracy: 0.0067
Epoch 3/25
636/636 [==============================] - 90s 141ms/step - loss: 0.3577 - accuracy: 0.9403 - val_loss: 5.7730 - val_accuracy: 0.0049
Epoch 4/25
636/636 [==============================] - 89s 140ms/step - loss: 0.2354 - accuracy: 0.9617 - val_loss: 6.0051 - val_accuracy: 0.0043
Epoch 5/25
 29/636 [>.............................] - ETA: 1:21 - loss: 0.1602 - accuracy: 0.9806

Process ForkPoolWorker-116:
Process ForkPoolWorker-106:
Process ForkPoolWorker-120:
Process ForkPoolWorker-110:
Process ForkPoolWorker-111:
Process ForkPoolWorker-113:
Process ForkPoolWorker-112:
Process ForkPoolWorker-114:
Process ForkPoolWorker-118:
Process ForkPoolWorker-108:
Process ForkPoolWorker-105:
Process ForkPoolWorker-109:
Process ForkPoolWorker-117:
Process ForkPoolWorker-119:
Process ForkPoolWorker-115:
Process ForkPoolWorker-107:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-104:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback

KeyboardInterrupt: 

  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/li